# 100 Prisoners Problem Simulation

100명의 죄수가 각자의 번호를 100개의 상자 중에서 찾아야 합니다. 각 죄수는 최대 50개의 상자만 열 수 있습니다.

- **Random 전략**: 무작위로 50개 상자 선택
- **Optimal 전략**: 자기 번호 상자부터 시작해서 루프를 따라감

---
## 1. 절차적 방식 (Procedural)

In [5]:
import random

In [6]:
def play_random(n):
    # using 0-99 instead of ranges 1-100
    pardoned = 0
    in_drawer = list(range(100))
    sampler = list(range(100))
    for _round in range(n):
        random.shuffle(in_drawer)
        found = False
        for prisoner in range(100):
            found = False
            for reveal in random.sample(sampler, 50):
                card = in_drawer[reveal]
                if card == prisoner:
                    found = True
                    break
            if not found:
                break
        if found:
            pardoned += 1
    return pardoned / n * 100   # %

In [7]:
def play_optimal(n):
    # using 0-99 instead of ranges 1-100
    pardoned = 0
    in_drawer = list(range(100))
    for _round in range(n):
        random.shuffle(in_drawer)
        for prisoner in range(100):
            reveal = prisoner
            found = False
            for go in range(50):
                card = in_drawer[reveal]
                if card == prisoner:
                    found = True
                    break
                reveal = card
            if not found:
                break
        if found:
            pardoned += 1
    return pardoned / n * 100   # %

In [8]:
n = 100_000
print(" Simulation count:", n)
print(f" Random play wins: {play_random(n):4.1f}% of simulations")
print(f"Optimal play wins: {play_optimal(n):4.1f}% of simulations")

 Simulation count: 100000
 Random play wins:  0.0% of simulations
Optimal play wins: 31.3% of simulations


---
## 2. 객체지향 방식 (OOP)

| 전략 | 설명 |
|------|------|
| `play_naive` | 무작위 선택 (중복 허용) |
| `play_naive_mem` | 무작위 선택 (중복 불허) |
| `play_optimum` | 루프 추적 전략 |

In [9]:
class PrisionersGame:
    """100 Prisoners Problem Game"""
    def __init__(self, num_drawers):
        assert num_drawers % 2 == 0
        self.num_drawers = num_drawers
        self.max_attempts = int(self.num_drawers / 2)
        self.drawer_ids = list(range(1, num_drawers + 1))
        shuffled = self.drawer_ids[:]
        random.shuffle(shuffled)
        self.drawers = dict(zip(self.drawer_ids, shuffled))

    def play_naive(self, player_number):
        """ Randomly open drawers """
        for attempt in range(self.max_attempts):
            if self.drawers[random.choice(self.drawer_ids)] == player_number:
                return True
        return False

    def play_naive_mem(self, player_number):
        """ Randomly open drawers but avoiding repetitions """
        not_attemped = self.drawer_ids[:]
        for attempt in range(self.max_attempts):
            guess = random.choice(not_attemped)
            not_attemped.remove(guess)
            if self.drawers[guess] == player_number:
                return True
        return False

    def play_optimum(self, player_number):
        """ Open the drawer that matches the player number and then open the drawer
        with the revealed number.
        """
        prev_attempt = player_number
        for attempt in range(self.max_attempts):
            if self.drawers[prev_attempt] == player_number:
                return True
            else:
                prev_attempt = self.drawers[prev_attempt]
        return False

    @classmethod
    def victory(cls, results):
        """Defines a victory of a game: all players won"""
        return all(results)

    approaches = [play_naive, play_naive_mem, play_optimum]

    def play(self, approach):
        """Plays this game and returns a list of booleans with
        True if a player won, False otherwise"""
        return [approach(self, player) for player in self.drawer_ids]

In [10]:
NUM_DRAWERS = 100
NUM_REPETITIONS = 100_000

print('{:15}: {:>6} ({})'.format('approach', 'wins', 'ratio'))
print('-' * 35)
for approach in PrisionersGame.approaches:
    num_victories = 0
    for _ in range(NUM_REPETITIONS):
        game = PrisionersGame(NUM_DRAWERS)
        num_victories += PrisionersGame.victory(game.play(approach))

    print('{:15}: {:>6} ({:.2%})'.format(
        approach.__name__, num_victories, num_victories / NUM_REPETITIONS))

approach       :   wins (ratio)
-----------------------------------
play_naive     :      0 (0.00%)
play_naive_mem :      0 (0.00%)
play_optimum   :  31232 (31.23%)


---
## 비교 요약

| 항목 | 절차적 방식 | OOP 방식 |
|------|------------|----------|
| 인덱싱 | 0-99 | 1-100 |
| 전략 수 | 2개 | 3개 |
| 실행 속도 | 빠름 | 객체 생성 오버헤드 |
| 확장성 | 제한적 | 새 전략 추가 용이 |